# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import config
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect(f"host=127.0.0.1 dbname=sparkifydb user={config.user}")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data/')

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

artist_id  artist_latitude  artist_location  artist_longitude  \
0  AR7G5I41187FB4CE6C              NaN  London, England               NaN   

  artist_name   duration  num_songs             song_id            title  year  
0    Adam Ant  233.40363          1  SONHOTT12A8C13493C  Something Girls  1982

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id', 'title','artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
df.columns

Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

In [11]:
artist_data = df[['artist_id', 'artist_name', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
log_files = get_files('data/log_data/')

In [49]:
filepath_log = log_files[0]

In [93]:
df = pd.read_json(filepath_log, lines=True)
df.tail()

artist        auth firstName gender  itemInSession lastName  \
184               None  Logged Out      None   None              0     None   
185               None   Logged In     Emily      F              1   Benson   
186               None   Logged In     Emily      F              2   Benson   
187               None  Logged Out      None   None              3     None   
188  Alanis Morissette   Logged In   Stefany      F              0    White   

        length level                        location method      page  \
184        NaN  paid                            None    PUT     Login   
185        NaN  paid  Augusta-Richmond County, GA-SC    GET      Home   
186        NaN  paid  Augusta-Richmond County, GA-SC    PUT    Logout   
187        NaN  paid                            None    GET      Home   
188  175.82975  free                     Lubbock, TX    PUT  NextSong   

     registration  sessionId                            song  status  \
184           NaN        253                            None     307   
185  1.540907e+12        253                            None     200   
186  1.540907e+12        253                            None     307   
187           NaN        253                            None     200   
188  1.540708e+12        186  Right Through You (LP Version)     200   

                ts                                          userAgent userId  
184  1541373796796                                               None         
185  1541373956796  "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...     58  
186  1541373957796  "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...     58  
187  1541373965796                                               None         
188  1541375113796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     83

In [94]:
df.shape

(189, 18)

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [95]:
# filtering for next song action because other actions are actions on the site (login, homepage)
df = df.loc[df['page'] == 'NextSong'].reset_index()
df.head(2)

index              artist       auth firstName gender  itemInSession  \
0      1  Professor Longhair  Logged In       Ann      F              0   
1      3          Gary Hobbs  Logged In    Jahiem      M              1   

  lastName     length level                       location method      page  \
0    Banks  214.20363  free             Salt Lake City, UT    PUT  NextSong   
1    Miles  245.52444  free  San Antonio-New Braunfels, TX    PUT  NextSong   

   registration  sessionId           song  status             ts  \
0  1.540896e+12        124  Mean Ol'World     200  1541292603796   
1  1.540817e+12         42    En Mi Mundo     200  1541300092796   

                                           userAgent userId  
0  Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...     99  
1  "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....     43

In [96]:
df.shape

(144, 19)

In [97]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-04 00:50:03.796
1   2018-11-04 02:54:52.796
2   2018-11-04 02:58:57.796
3   2018-11-04 03:02:20.796
4   2018-11-04 04:35:52.796
Name: ts, dtype: datetime64[ns]

In [101]:
t.hour

AttributeError: 'Series' object has no attribute 'hour'

In [86]:
t[1].weekday()

6

In [98]:
# list of timestamp
time_data = [t[1].timestamp(), t[1].hour, t[1].day, t[1].weekofyear, t[1].month, t[1].year, t[1].weekday()]
column_labels = ['timestamp', 'hour', 'day', 'week_of_year', 'month', 'year', 'weekday']

In [100]:
dict(zip(column_labels, time_data))

{'timestamp': 1541300092.796,
 'hour': 2,
 'day': 4,
 'week_of_year': 44,
 'month': 11,
 'year': 2018,
 'weekday': 6}

In [ ]:
time_df = 
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    songid, artistid = results if results else None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [78]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.